## Import

In [1]:
import torch
import numpy as np
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchsummary import summary
from PIL import Image
import os
import re
import nibabel as nib

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data Load

In [3]:
zip_path = sorted(glob('/content/drive/Shareddrives/캡스톤 디자인1/dataset/original_512/*'))

In [4]:
zip_path

['/content/drive/Shareddrives/캡스톤 디자인1/dataset/original_512/images_01.zip',
 '/content/drive/Shareddrives/캡스톤 디자인1/dataset/original_512/images_02.zip',
 '/content/drive/Shareddrives/캡스톤 디자인1/dataset/original_512/images_03.zip',
 '/content/drive/Shareddrives/캡스톤 디자인1/dataset/original_512/images_04.zip',
 '/content/drive/Shareddrives/캡스톤 디자인1/dataset/original_512/images_05.zip',
 '/content/drive/Shareddrives/캡스톤 디자인1/dataset/original_512/images_06.zip',
 '/content/drive/Shareddrives/캡스톤 디자인1/dataset/original_512/images_07.zip',
 '/content/drive/Shareddrives/캡스톤 디자인1/dataset/original_512/images_08.zip',
 '/content/drive/Shareddrives/캡스톤 디자인1/dataset/original_512/images_09.zip',
 '/content/drive/Shareddrives/캡스톤 디자인1/dataset/original_512/images_10.zip',
 '/content/drive/Shareddrives/캡스톤 디자인1/dataset/original_512/images_11.zip',
 '/content/drive/Shareddrives/캡스톤 디자인1/dataset/original_

In [5]:
import zipfile
for path in zip_path:
  zip_file = zipfile.ZipFile(path)
  zip_file.extractall('/content/datasets/'+path.split('/')[-1].split('.')[0])
  zip_file.close()

In [6]:
PATH = '/content/datasets/'
from glob import glob
file_data = glob(PATH+'*')

In [7]:
file_data[0]

'/content/datasets/images_12'

In [8]:
dummy=glob('/content/drive/MyDrive/capstone/x-ray_image/images_01/images/*')

In [9]:
IM = glob('/content/datasets/images*/')
IM

['/content/datasets/images_12/',
 '/content/datasets/images_06/',
 '/content/datasets/images_08/',
 '/content/datasets/images_11/',
 '/content/datasets/images_05/',
 '/content/datasets/images_02/',
 '/content/datasets/images_09/',
 '/content/datasets/images_03/',
 '/content/datasets/images_01/',
 '/content/datasets/images_10/',
 '/content/datasets/images_07/',
 '/content/datasets/images_04/']

In [10]:
my_glob = glob('/content/datasets/images*/*.png')
print('Number of Observations: ', len(my_glob))

Number of Observations:  76666


## Data Load

In [11]:
import pandas as pd
df= pd.read_csv('/content/drive/Shareddrives/캡스톤 디자인1/processed_data_entry.csv')

In [12]:
df

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
0,00000001_000.png,Cardiomegaly,0,1,57,M,PA,2682,2749,0.143,0.143
1,00000001_001.png,"Cardiomegaly,Emphysema",1,1,58,M,PA,2894,2729,0.143,0.143
2,00000001_002.png,"Cardiomegaly,Effusion",2,1,58,M,PA,2500,2048,0.168,0.168
3,00000002_000.png,No Finding,0,2,80,M,PA,2500,2048,0.171,0.171
4,00000003_001.png,Hernia,0,3,74,F,PA,2500,2048,0.168,0.168
...,...,...,...,...,...,...,...,...,...,...,...
76661,00030801_001.png,"Mass,Pneumonia",1,30801,38,M,PA,2048,2500,0.168,0.168
76662,00030802_000.png,No Finding,0,30802,28,M,PA,2048,2500,0.168,0.168
76663,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168
76664,00030804_000.png,No Finding,0,30804,29,F,PA,2048,2500,0.168,0.168


In [13]:
df.drop(['Follow-up #', 'Patient ID',
       'Patient Age', 'Patient Gender', 'View Position', 'OriginalImage[Width','Height]', 'OriginalImagePixelSpacing[x', 'y]'],axis = 1,inplace = True)

In [14]:
df

,Image Index,Finding Labels
0,00000001_000.png,Cardiomegaly
1,00000001_001.png,"Cardiomegaly,Emphysema"
2,00000001_002.png,"Cardiomegaly,Effusion"
3,00000002_000.png,No Finding
4,00000003_001.png,Hernia
...,...,...
76661,00030801_001.png,"Mass,Pneumonia"
76662,00030802_000.png,No Finding
76663,00030803_000.png,No Finding
76664,00030804_000.png,No Finding


In [15]:
full_img_paths = {os.path.basename(x): x for x in my_glob}
df['full_path'] = df['Image Index'].map(full_img_paths.get)

In [16]:
df

,Image Index,Finding Labels,full_path
0,00000001_000.png,Cardiomegaly,/content/datasets/images_01/00000001_000.png
1,00000001_001.png,"Cardiomegaly,Emphysema",/content/datasets/images_01/00000001_001.png
2,00000001_002.png,"Cardiomegaly,Effusion",/content/datasets/images_01/00000001_002.png
3,00000002_000.png,No Finding,/content/datasets/images_01/00000002_000.png
4,00000003_001.png,Hernia,/content/datasets/images_01/00000003_001.png
...,...,...,...
76661,00030801_001.png,"Mass,Pneumonia",/content/datasets/images_12/00030801_001.png
76662,00030802_000.png,No Finding,/content/datasets/images_12/00030802_000.png
76663,00030803_000.png,No Finding,/content/datasets/images_12/00030803_000.png
76664,00030804_000.png,No Finding,/content/datasets/images_12/00030804_000.png


In [17]:
classes = ['Atelectasis',
                'Consolidation',
                'Infiltration', 
                'Pneumothorax', 
                'Edema', 
                'Emphysema',
                'Fibrosis', 
                'Effusion',
                'Pneumonia',
                'Pleural_Thickening',
                'Cardiomegaly',
                'Nodule', 
                'Mass', 
                'Hernia']
for label in classes:
  df[label] = df['Finding Labels'].map(lambda result: 1.0 if label in result else 0)
  # lambda x: (if문이 참일 때 x에 적용하고 싶은 효과) if....else(거짓일 때 적용하고 싶은 효과), iterable 객체

In [18]:
df
# 각 이미지에서 질병들의 유무에 따라 1또는 0으로 표현(one hot encoding)

,Image Index,Finding Labels,full_path,Atelectasis,Consolidation,Infiltration,Pneumothorax,Edema,Emphysema,Fibrosis,Effusion,Pneumonia,Pleural_Thickening,Cardiomegaly,Nodule,Mass,Hernia
0,00000001_000.png,Cardiomegaly,/content/datasets/images_01/00000001_000.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,00000001_001.png,"Cardiomegaly,Emphysema",/content/datasets/images_01/00000001_001.png,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,00000001_002.png,"Cardiomegaly,Effusion",/content/datasets/images_01/00000001_002.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,00000002_000.png,No Finding,/content/datasets/images_01/00000002_000.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00000003_001.png,Hernia,/content/datasets/images_01/00000003_001.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76661,00030801_001.png,"Mass,Pneumonia",/content/datasets/images_12/00030801_001.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
76662,00030802_000.png,No Finding,/content/datasets/images_12/00030802_000.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76663,00030803_000.png,No Finding,/content/datasets/images_12/00030803_000.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76664,00030804_000.png,No Finding,/content/datasets/images_12/00030804_000.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
data = df[classes]
df['labels'] = data.apply(lambda row: np.argmax(row) if np.sum(row)>0 else -1, axis = 1)

In [20]:
df['labels']
# 각 어떤 인덱스의 질병을 가지고 있는지에 대한 정보

0        10
1         5
2         7
3        -1
4        13
         ..
76661     8
76662    -1
76663    -1
76664    -1
76665    -1
Name: labels, Length: 76666, dtype: int64

In [21]:
df = df[df['Finding Labels']!= 'No Finding']
df.reset_index(drop = True, inplace = True)

In [22]:
target_cols = df.drop(['Image Index','Finding Labels', 'full_path','labels'], axis=1).columns.to_list()

In [23]:
target_cols

['Atelectasis',
 'Consolidation',
 'Infiltration',
 'Pneumothorax',
 'Edema',
 'Emphysema',
 'Fibrosis',
 'Effusion',
 'Pneumonia',
 'Pleural_Thickening',
 'Cardiomegaly',
 'Nodule',
 'Mass',
 'Hernia']

In [24]:
columns = df.columns[3:-1]
columns

Index(['Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema',
       'Emphysema', 'Fibrosis', 'Effusion', 'Pneumonia', 'Pleural_Thickening',
       'Cardiomegaly', 'Nodule', 'Mass', 'Hernia'],
      dtype='object')

In [25]:
from sklearn.model_selection import train_test_split
paths = df['full_path'].values
columns = df.columns[3:-1]
labels = df[columns].values
(train_path, val_path ,train_labels , val_labels) = train_test_split(paths,labels,test_size = 0.2,random_state = 42)
#orginal dataset에서 특정 클래스 비율이 불균형하기 때문에 stratify를 지정하게 되면 사용하지 않는 이유

In [26]:
len(paths)

51759

In [27]:
## 훈련데이터세트의 shape 
print(paths.shape)
#(51759,) 

## 훈련세트, 테스트세트 shape 
print(train_path.shape, val_path.shape)
# (41407,) (10352,)

(51759,)
(41407,) (10352,)


In [59]:
np.where(val_labels[1]==1)[0].tolist()

[7, 11, 12]

In [28]:
class Train_Dataset(Dataset):
    def __init__(self, data_path,labels, transform = None):
        self.data_path = data_path
        self.transform = transform
        self.labels = labels
    def __len__(self):
        return len(self.data_path)
    def __getitem__(self,idx):
        path = self.data_path[idx]
        img = np.array(Image.open(path))
        img = img/255
        img = img[:,:,np.newaxis] 
        label = self.labels[idx]
        if self.transform is not None:
            transformed = self.transform(image=img)
            image = transformed['image']
        return image, label

In [29]:
class val_Dataset(Dataset):
    def __init__(self, data_path,labels, transform = None):
        self.data_path = data_path
        self.transform = transform
        self.labels = labels
    def __len__(self):
        return len(self.data_path)
    def __getitem__(self,idx):
        path = self.data_path[idx]
        img = np.array(Image.open(path))
        img = img/255
        img = img[:,:,np.newaxis]    
        label = self.labels [idx]
        if self.transform is not None:
            transformed = self.transform(image=img)
            image = transformed['image']
        return image, label

In [30]:
#!pip install --upgrade --force-reinstall --no-deps albumentations
!pip install albumentations==0.4.6

     |████████████████████████████████| 117 kB 4.3 MB/s 
     |████████████████████████████████| 948 kB 28.1 MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65174 sha256=3045a6b58009675d7dbc618a36f1561ea9965b76f4e5d4f75f825016c0cc3b01
  Stored in directory: /root/.cache/pip/wheels/cf/34/0f/cb2a5f93561a181a4bcc84847ad6aaceea8b5a3127469616cc
Successfully built albumentations
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [31]:
import albumentations as A                                                                           
from albumentations.pytorch import ToTensorV2

In [32]:
data_transforms = {
    'train': A.Compose(
    [
     ToTensorV2()
     ]
    ),
    'val': A.Compose(
        [
     ToTensorV2()
                            ]
                           )
}
'''
A.RandomCrop(224, 224),
     A.OneOf([
              A.HorizontalFlip(p=1),
              A.RandomRotate90(p=1),
              A.VerticalFlip(p=1)            
    ], p=1), 
'''

'\nA.RandomCrop(224, 224),\n     A.OneOf([\n              A.HorizontalFlip(p=1),\n              A.RandomRotate90(p=1),\n              A.VerticalFlip(p=1)            \n    ], p=1), \n'

In [60]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]

# Number of classes in the dataset
num_classes = 14

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 30

In [61]:
train_data = Train_Dataset(train_path,train_labels,transform = data_transforms['train'])
val_data = val_Dataset(val_path,val_labels,transform =  data_transforms['val'])

In [62]:
image_datasets = {'train' : train_data , 'val' : val_data}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True) for x in ['train', 'val']}
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Model

In [63]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [64]:
pip install timm

In [65]:
import timm

In [66]:
#model = timm.create_model('vgg16', pretrained=True, num_classes = 14)

In [67]:
#model.features[0] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [68]:
model = torch.load('/content/drive/Shareddrives/캡스톤 디자인1/codes/Densenet169_multilabelclassfication_41_45epoch.pt')

RuntimeError: ignored

In [69]:
device = torch.device('cuda:0')
model.features.conv0 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNormAct2d(
      64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (act): ReLU(inplace=True)
    )
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): DenseBlock(
      (denselayer1): DenseLayer(
        (norm1): BatchNormAct2d(
          64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (act): ReLU(inplace=True)
        )
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNormAct2d(
          128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (act): ReLU(inplace=True)
        )
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): DenseLayer(
        (norm1): BatchNormAct2d(
          96, eps=1

In [70]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score , roc_curve , accuracy_score,classification_report,multilabel_confusion_matrix
import tqdm

In [71]:
import torch.optim as optim
from torch.optim import lr_scheduler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft = model.to(device)

criterion = nn.BCEWithLogitsLoss()

# specify optimizer
optimizer_ft = optim.Adam(model.parameters(), lr=0.00001)
sgdr_partial = lr_scheduler.CosineAnnealingLR(optimizer_ft, T_max=5, eta_min=0.005 )

In [72]:
def train_model(model, dataloaders, criterion, optimizer, scheduler , num_epochs=25):
    since = time.time()
    train_acc_history = []
    train_loss_hist = [] 
    train_classification_report =[]
    train_confusion_matrix = []
    val_loss_hist = []
    val_acc_history = [] 
    val_classification_report =[]
    val_confusion_matrix = []
    epoch_train_label = []
    epoch_train_output = []
    epoch_val_label = []
    epoch_val_output = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            start = time.time()
            print('phase : %r' %phase)
            if phase == 'train':
                model.train() # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_acc = 0
            running_precision =0
            running_recall = 0
            running_f1_score =0 
            running_auc = 0
            # Iterate over data.
            full_label = []
            full_output = []
            full_prediction = []
            for inputs, labels in tqdm.notebook.tqdm(dataloaders[phase]):
                inputs = inputs.to(device = device, dtype = torch.float32)
                labels = labels.to(device = device)
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    outputs = model(inputs)
                    loss = criterion(outputs,labels.float())
                    pred = nn.Sigmoid()(outputs)
                    full_prediction += pred.detach().cpu().numpy().tolist()
                    pred[pred <0.5] = 0
                    pred[pred >=0.5] = 1
                    if phase == 'train':
                    # backward + optimize only if in training phase
                        loss.backward()
                        optimizer.step()
                        scheduler.step()
                # statistics
                running_loss += loss.item() * inputs.size(0)
                full_label+= labels.detach().cpu().numpy().tolist()
                full_output+= pred.detach().cpu().numpy().tolist()
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = accuracy_score(full_label,full_output)
            epoch_classification_report = classification_report(full_label,full_output,target_names = target_cols)
            epoch_multilabel_confusion_matrix = multilabel_confusion_matrix(full_label,full_output)
             
            if phase == 'train':
              train_acc_history.append(epoch_acc)
              train_loss_hist.append(epoch_loss)
              epoch_train_label.append(full_label)
              epoch_train_output.append(full_prediction)
              train_classification_report.append(epoch_classification_report)
              train_confusion_matrix.append(epoch_multilabel_confusion_matrix)
            print('\n{} Loss: {:.4f} Acc: {:.4f}, Time : {:.4f}'.format(phase, epoch_loss ,epoch_acc,time.time()-start))
            #,   , , epoch_precision,epoch_recall, epoch_auc, epoch_f1_score 
            print('\n--------------------------------')
            print('|{}Epoch {}phase multilabel_confusion_matrix report|'.format(epoch+1,phase))
            print('--------------------------------')
            print('\n{}'.format(epoch_multilabel_confusion_matrix))
            print('\n--------------------------------')
            print('\n{}Epoch {}phase classification report'.format(epoch+1,phase))
            print('--------------------------------')
            print(epoch_classification_report)

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
                val_loss_hist.append(epoch_loss)
                epoch_val_label.append(full_label)
                epoch_val_output.append(full_prediction)
                val_classification_report.append(epoch_classification_report)
                val_confusion_matrix.append(multilabel_confusion_matrix)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    train_dict = {'Loss' : train_loss_hist , 'Accuarcy' : train_acc_history,'Classification_report' : train_classification_report,'Confusion_matrix' :train_confusion_matrix, 'train_label' : epoch_train_label , 'train_output' : epoch_train_output}
    #
    val_dict = {'Loss' :val_loss_hist , 'Accuarcy' : val_acc_history,'Classification_report' : val_classification_report,'val_label' : epoch_val_label , 'Confusion_matrix' :val_confusion_matrix,'val_output' : epoch_val_output}
    #, 
    return model, train_dict ,val_dict

In [73]:
#summary(model.to('cuda'),(3,512,512))

In [74]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
criterion = nn.BCEWithLogitsLoss()

# Train and evaluate
model_ft, train_dict, val_dict  = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft,sgdr_partial, num_epochs=num_epochs)

'''

confusion maxrix
[[true negative , false positive
false negative , true positive]]

[[tn , fp
fn , tp]]

'''

Epoch 1/30
----------
phase : 'train'


  0%|          | 0/5176 [00:00<?, ?it/s]

In [ ]:
torch.save(model_ft, '/content/drive/Shareddrives/캡스톤 디자인1/codes/Densenet169_multilabelclassfication_46_50epoch.pt')

In [ ]:
import pickle
# save data
with open('/content/drive/Shareddrives/캡스톤 디자인1/model_history/Densenet169_multilabelclassfication_train_dict_46_50epoch.pkl','wb') as fw:
    pickle.dump(train_dict, fw)

In [ ]:
with open('/content/drive/Shareddrives/캡스톤 디자인1/model_history/Densenet169_multilabelclassfication_val_dict_46_50epoch.pkl','wb') as ff:
    pickle.dump(val_dict, ff)

In [ ]:
import pickle
# load data
with open('/content/drive/Shareddrives/캡스톤 디자인1/model_history/Densenet169_multilabelclassfication_val_dict_21_25epoch.pkl','rb') as fr:
    val_dict = pickle.load(fr)

In [ ]:
with open('/content/drive/Shareddrives/캡스톤 디자인1/model_history/Densenet169_multilabelclassfication_train_dict_21_25epoch.pkl','rb') as fr:
    train_dict = pickle.load(fr)

In [ ]:
train_dict.keys()

dict_keys(['Loss', 'Accuarcy', 'Classification_report', 'Confusion_matrix', 'train_label', 'train_output'])

In [ ]:
train_label=val_dict['train_label']
train_output = val_dict['train_output']

KeyError: ignored

In [ ]:
epoch_1_label = np.array(train_label[0])
epoch_1_output= np.array(train_output[0])
epoch_10_label = np.array(train_label[-1])
epcoh_10_output = np.array(train_output[-1])

In [ ]:
from sklearn import metrics
from sklearn.datasets import make_classification
from sklearn.metrics import roc_curve, auc, roc_auc_score
n_classes = 14
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(epoch_1_label[:, i], epoch_1_output[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot of a ROC curve for a specific class
plt.figure(figsize=(10, 10))
for idx, i in enumerate(range(n_classes)):
    plt.plot(fpr[i], tpr[i], label=' {:} ROC curve (area = {:0.2f})'.format(classes[i] ,roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Densenet169 Epoch_1_ROC curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
from sklearn import metrics
from sklearn.datasets import make_classification
from sklearn.metrics import roc_curve, auc, roc_auc_score
n_classes = 14
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(epoch_10_label[:, i], epcoh_10_output[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot of a ROC curve for a specific class
plt.figure(figsize=(10, 10))
for idx, i in enumerate(range(n_classes)):
    plt.plot(fpr[i], tpr[i], label=' {:} ROC curve (area = {:0.2f})'.format(classes[i] ,roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Densenet169 Epoch_10_ROC curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
train_loss = []
val_loss = []
for i in range(len(train_dict['Loss'])):
  train_loss.append(train_dict['Loss'][i])
for i in range(len(val_dict['Loss'])):
  val_loss.append(val_dict['Loss'][i])

In [ ]:
plt.plot(train_loss)
plt.plot(val_loss)
plt.title('Densenet169 Loss', fontsize = 15)
plt.xlabel('Epoch', fontsize = 15)
plt.ylabel('Loss', fontsize = 15)
plt.ylim(0,0.5)
plt.legend(['train','test'], loc='upper left')
plt.show()

In [ ]:
model_ft = torch.load('/content/drive/Shareddrives/캡스톤 디자인1/codes/resnet151_multiclassclassfication_nofinding_제외_timm.pt')

In [ ]:
model_ft = model_ft.to(device)


# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters(),lr=0.001)# 업데이트 할 파라미터만 넣어준다.
class test_Dataset(Dataset):
    def __init__(self, data_path,transform = None):
        self.data_path = data_path
        self.transform = transform
    def __len__(self):
        return len(self.data_path)
    def __getitem__(self,idx):
        path = self.data_path[idx]
        img = np.array(Image.open(path))
        img = img[:,:,np.newaxis]
        img = img/255
        label =  class2idx[df[df['Image Index']== path.split('/')[-1]]['Finding Labels'].values[0]]
        if self.transform is not None:
            transformed = self.transform(image=img)
            image = transformed['image']
        return image.float(), label

In [ ]:
data_transforms_test = A.Compose(
        [
     ToTensorV2()
     ]
     )

In [ ]:
test_data = test_Dataset(val_path,transform = data_transforms_test)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle= False)

In [ ]:
summary(model_ft.to('cuda'),(1,512,512))

In [ ]:
running_loss = 0.0
running_corrects = 0
start = time.time()
output_list = list()
criterion = nn.CrossEntropyLoss()

for inputs, labels in test_loader:
    inputs = inputs.to(device = device, dtype = torch.float32)
    labels = labels.to(device = device)
    # zero the parameter gradients
    optimizer_ft.zero_grad()
    outputs = model_ft(inputs)
    output_list+=list((torch.argmax(outputs,axis = 1).cpu().numpy()))
    loss = criterion(outputs, labels)
    _, preds = torch.max(outputs, 1)
    running_loss += loss.item() * inputs.size(0)
    running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(test_loader.dataset)
    epoch_acc = running_corrects.double() / len(test_loader.dataset)
print('Test Loss: {:.4f} Acc: {:.4f}, Time : {:.4f}'.format(epoch_loss, epoch_acc, time.time()-start))

In [ ]:
len(test_loader)*2

In [ ]:
labels

In [ ]:
torch.argmax(outputs,axis = 1).cpu().numpy()

In [ ]:
outputs.shape

In [ ]:
dummy_df = pd.DataFrame(output_list)

In [ ]:
dummy_df.value_counts()

In [ ]:
val_df['Finding Labels'].value_counts()

In [ ]:
class2idx

In [ ]:
#레이어 시각화

for w in model_ft.parameters():
    w = w.data.cpu()
    print(w.shape)
    break

# normalize weights
min_w = torch.min(w)
w1 = (-1/(2 * min_w)) * w + 0.5

# make grid to display it
grid_size = len(w1)
x_grid = [w1[i] for i in range(grid_size)]
x_grid = torchvision.utils.make_grid(x_grid, nrow=8, padding=1)

plt.figure(figsize=(10, 10))
plt.imshow(x_grid.permute(2,1,0))

In [ ]:
sample_image = np.array(Image.open(train_path[0]))
sample_image = sample_image[:,:,np.newaxis]/255
transform = transforms.Compose([
    transforms.ToTensor()
])
sample_image = transform(sample_image)
sample_image=sample_image.unsqueeze(0)

In [ ]:
sample_image

In [ ]:
no_of_layers=0
conv_layers=[]

model_children=list(model_ft.children())

for child in model_children:
  if type(child)==nn.Conv2d:
    no_of_layers+=1
    conv_layers.append(child)
  elif type(child)==nn.Sequential:
    for layer in child.children():
      for bottleneck in layer.children():
        if type(bottleneck)==nn.Conv2d:
          no_of_layers+=1
          conv_layers.append(bottleneck)
print(no_of_layers)

In [ ]:
results = [conv_layers[0](sample_image.to(device = device, dtype = torch.float32))]
for i in range(1, len(conv_layers)):
    results.append(conv_layers[i](results[-1]))
outputs = results

In [ ]:
# visualize 8 features map from each layer 
for num_layer in range(len(results)):
    plt.figure(figsize=(50, 10))
    layer_viz = results[num_layer][0, :, :, :]
    layer_viz = layer_viz.data
    print("Layer ",num_layer+1)
    for i, filter in enumerate(layer_viz):
        if i == 16: 
            break
        plt.subplot(2, 8, i + 1)
        plt.imshow(filter.cpu().numpy(), cmap='gray')
        plt.axis("off")
    plt.show()
    plt.close()